# Running cellpose 2.0 in colab with a GPU
# TRAINING MODEL ONLY

<font size = 4>Cellpose 2.0 now allows you to train your own models in the GUI!

In this notebook, you can **train** a custom model using your labels (`_seg.npy`) files, or other labels as `_masks.tif` files. If you already have a trained model go to the other notebook **run custom model**

For more details on cellpose 2.0 check out the [paper](https://www.biorxiv.org/content/10.1101/2022.04.01.486764v1) or the [talk](https://www.youtube.com/watch?v=3ydtAhfq6H0).

Mount your google drive to access all your image files, segmentations, and custom models. This also ensures that any models you train are saved to your google drive.

This notebook was inspired by the Zero-Cost Deep-Learning to Enhance Microscopy project (https://github.com/HenriquesLab/DeepLearning_Collab/wiki). Jointly developed by the Jacquemet (link to https://cellmig.org/) and Henriques (https://henriqueslab.github.io/) laboratories. Please check out their great work!

# Setup

We will first install cellpose 2.0, check the GPU is working, and mount google drive to get your models and images.

## Installation

Install cellpose -- by default the torch GPU version is installed in COLAB notebook.

In [20]:
!pip install "opencv-python-headless<4.3"


Check CUDA version and that GPU is working in cellpose and import other libraries.

In [21]:
!nvcc --version
!nvidia-smi

import os, shutil
import numpy as np
import matplotlib.pyplot as plt
from cellpose import core, utils, io, models, metrics
from glob import glob
use_GPU = core.use_gpu()

'nvcc' is not recognized as an internal or external command,
operable program or batch file.


Fri Jul 21 14:16:06 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.79                 Driver Version: 531.79       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti    WDDM | 00000000:01:00.0  On |                  N/A |
|  0%   35C    P8               12W / 225W|    873MiB /  8192MiB |     11%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Train model on manual annotations

Skip this if you already have a pretrained model.

## Fill out the form below with the paths to your data and the parameters to start training.

<font size=4>**`You do not need to change any parameter. Just the path of the training data and the model name`**

## Training parameters

<font size = 4> **Paths for training, predictions and results**


<font size = 4>**`train_dir:`, `test_dir`:** These are the paths to your folders train_dir (with images and masks of training images) and test_dir (with images and masks of test images). You can leave the test_dir blank, but it's recommended to have some test images to check the model's performance. To find the paths of the folders containing the respective datasets, go to your Files on the left of the notebook, navigate to the folder containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.

<font size = 4>**`initial_model`:** Choose a model from the cellpose [model zoo](https://cellpose.readthedocs.io/en/latest/models.html#model-zoo) to start from.

<font size = 4>**`model_name`**: Enter the path where your model will be saved once trained (for instance your result folder).

<font size = 4>**Training parameters**

<font size = 4>**`number_of_epochs`:** Input how many epochs (rounds) the network will be trained. At least 100 epochs are recommended, but sometimes 250 epochs are necessary, particularly from scratch. **Default value: 100**

## OUTPUT
<font size =4>**`The model will be saved in the model folder which will be in the training folder of your images`**



In [22]:
#@markdown ###Path to images and masks:

train_dir = input("Enter the path of the folder that contains the training data (image file + _seg.npy file)") #@param {type:"string"}
train_dir=train_dir.replace('\\', '/')

# This is facultative
test_dir = "" #@param {type:"string"}

#Define where the patch file will be saved
base = "/content"

# model name and path
#@markdown ###Name of the pretrained model to start from and new model name:
from cellpose import models
initial_model = "cyto" #@param ['cyto','nuclei','tissuenet','livecell','cyto2','CP','CPx','TN1','TN2','TN3','LC1','LC2','LC3','LC4','scratch']

#
model_name = input("What will be the name of the model/model file")   #@param {type:"string"}

# other parameters for training.
#@markdown ###Training Parameters:
#@markdown Number of epochs:
n_epochs =  100#@param {type:"number"}

Channel_to_use_for_training = "Grayscale" #@param ["Grayscale", "Blue", "Green", "Red"]

# @markdown ###If you have a secondary channel that can be used for training, for instance nuclei, choose it here:

Second_training_channel= "None" #@param ["None", "Blue", "Green", "Red"]


#@markdown ###Advanced Parameters

Use_Default_Advanced_Parameters = True #@param {type:"boolean"}
#@markdown ###If not, please input:
learning_rate = 0.1 #@param {type:"number"}
weight_decay = 0.0001 #@param {type:"number"}

if (Use_Default_Advanced_Parameters):
  print("Default advanced parameters enabled")
  learning_rate = 0.1
  weight_decay = 0.0001

#here we check that no model with the same name already exist, if so delete
model_path = train_dir + 'models/'
if os.path.exists(model_path+'/'+model_name):
  print("!! WARNING: "+model_name+" already exists and will be deleted in the following cell !!")

if len(test_dir) == 0:
  test_dir = None

# Here we match the channel to number
if Channel_to_use_for_training == "Grayscale":
  chan = 0
elif Channel_to_use_for_training == "Blue":
  chan = 3
elif Channel_to_use_for_training == "Green":
  chan = 2
elif Channel_to_use_for_training == "Red":
  chan = 1


if Second_training_channel == "Blue":
  chan2 = 3
elif Second_training_channel == "Green":
  chan2 = 2
elif Second_training_channel == "Red":
  chan2 = 1
elif Second_training_channel == "None":
  chan2 = 0

if initial_model=='scratch':
  initial_model = 'None'

Default advanced parameters enabled


## Train new model

Using settings from form above, train model in notebook.

In [23]:
# start logger (to see training across epochs)
print(train_dir)
logger = io.logger_setup()

# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=use_GPU, model_type=initial_model)

# set channels
channels = [chan, chan2]

# get files
output = io.load_train_test_data(train_dir, test_dir, mask_filter='_seg.npy')
train_data, train_labels, _, test_data, test_labels, _ = output

new_model_path = model.train(train_data, train_labels,
                              test_data=test_data,
                              test_labels=test_labels,
                              channels=channels,
                              save_path=train_dir,
                              n_epochs=n_epochs,
                              learning_rate=learning_rate,
                              weight_decay=weight_decay,
                              nimg_per_epoch=8,
                              model_name=model_name)

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

##executed in 1 min for 6 images

C:/Users/JpRas/Downloads/train+model_Viri-20230718T174006Z-001/train+model_Viri/train
creating new log file
2023-07-21 14:16:10,910 [INFO] WRITING LOG OUTPUT TO C:\Users\JpRas\.cellpose\run.log
2023-07-21 14:16:10,910 [INFO] 
cellpose version: 	2.2.1 
platform:       	win32 
python version: 	3.8.17 
torch version:  	2.0.1
2023-07-21 14:16:10,911 [INFO] >> cyto << model set to be used
2023-07-21 14:16:10,912 [INFO] ** TORCH CUDA version installed and working. **
2023-07-21 14:16:10,913 [INFO] >>>> using GPU
2023-07-21 14:16:11,066 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2023-07-21 14:16:11,068 [INFO] not all flows are present, running flow generation for all images
2023-07-21 14:16:11,271 [INFO] 6 / 6 images in C:/Users/JpRas/Downloads/train+model_Viri-20230718T174006Z-001/train+model_Viri/train folder have labels
2023-07-21 14:16:11,451 [INFO] computing flows for labels


100%|██████████| 6/6 [00:00<00:00, 12.63it/s]


2023-07-21 14:16:12,035 [INFO] >>>> median diameter set to = 30
2023-07-21 14:16:12,035 [INFO] >>>> mean of training label mask diameters (saved to model) 26.061
2023-07-21 14:16:12,038 [INFO] >>>> training network with 2 channel input <<<<
2023-07-21 14:16:12,039 [INFO] >>>> LR: 0.10000, batch_size: 8, weight_decay: 0.00010
2023-07-21 14:16:12,039 [INFO] >>>> ntrain = 6
2023-07-21 14:16:12,040 [INFO] >>>> nimg_per_epoch = 8
2023-07-21 14:16:17,028 [INFO] Epoch 0, Time  5.0s, Loss 2.5009, LR 0.0000
2023-07-21 14:16:17,214 [INFO] saving network parameters to C:/Users/JpRas/Downloads/train+model_Viri-20230718T174006Z-001/train+model_Viri/train\models/test
2023-07-21 14:16:18,002 [INFO] Epoch 5, Time  6.0s, Loss 1.0560, LR 0.0556
2023-07-21 14:16:18,955 [INFO] Epoch 10, Time  6.9s, Loss 0.1948, LR 0.1000
2023-07-21 14:16:20,828 [INFO] Epoch 20, Time  8.8s, Loss 0.3175, LR 0.1000
2023-07-21 14:16:22,711 [INFO] Epoch 30, Time 10.7s, Loss 0.2148, LR 0.1000
2023-07-21 14:16:24,576 [INFO] Epoc